**<h1>What can this version of Postman do?</h1>**



*   **Easily customize messages to reach a wide audience**: Create a message template, upload a file containing customization parameters, and we will handle the rest for you.
*   **View stats**: Keep track of your campaign's progress as it is sending and check back when it is completed.


**<h1>Does Postman send messages to everyone who subscribes to the bot?</h1>**

You can control who you contact through your Telegram bot by uploading the mobile number of the recipients. Postman converts the phone number you uploaded to Telegram user IDs and sends your message to Telegram bot subscribers.

**<h1>Getting started</h1>**

The document you are reading is not a static web page, but an interactive environment called a Colab notebook that lets you write and execute code.
If this is your first time using a notebook, please read the instructions carefully, don't skip me!

To execute the code in a cell, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut "Command/Ctrl+Enter".


---



In order for us to "connect" with our telegram bot through our python notebook, we have to install a package so that we can "talk" with the telegram Application Programming Interface (API)!

In [ ]:
# @title <h1>Install Packages</h1>
!pip install python-telegram-bot==20.6
!pip install psycopg2-binary==2.9.9
!pip install pandas==1.5.3
!pip install sqlalchemy==1.4.50

import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import telegram
from telegram import Bot
import asyncio
import logging
from time import time
from asyncio import sleep, Lock
from google.colab import output, userdata
from io import StringIO
from telegram.ext import ApplicationBuilder, BaseRateLimiter
from telegram.request import HTTPXRequest
output.clear()

<h1>Upload Recipient List</h1>

Find the icon on the left of your screen to upload your Recipient List

<div align="right">
    <img src="https://blog.roboflow.com/content/images/2020/10/upload.png" />
</div>

In [ ]:
# @title <h1>Input file name of recipient list</h1> { form-width: "50%" }
recipient_list_filename = "xxxxxx" # @param {type:"string"}
db_params = {
    'dbname':"postgres",
    'user':userdata.get('user'),
    'password':userdata.get('password'),
    'host':userdata.get('host'),
    'port':"port"
}

db_uri = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

engine = create_engine(db_uri)

query = "SELECT telegram_id, phone_number FROM subscriber_list"

subscriber_list = pd.read_sql(query, engine)

subscriber_list['phone_number'] = subscriber_list['phone_number'].apply(lambda x: str(x).replace("-", "").strip())
subscriber_list.rename(columns={'phone_number': 'Mobile No'}, inplace=True)

#read recepient spreadsheet
df_recipient_list = pd.read_excel(recipient_list_filename)
df_recipient_list['Mobile No'] = df_recipient_list['Mobile No'].apply(lambda x: str(x).replace("-", "").strip())

# Join the dataframes on the phone_numbers column
merged_df = pd.merge(df_recipient_list, subscriber_list, on='Mobile No', how='inner')

<h1>Create Message Template</h1>

To personalise your message, include keywords that are surrounded by {curly braces}. <u>**The keywords in your message template should match the headers in your recipients excel file.**</u>

```
Dear {Name}, your mobile number is {phone_number} and your telegram ID is {telegram_id}.
```

To format your message, refer to the following:

```
<b>bold</b>

<i>italic</i>

<u>underline</u>

<s>strikethrough</s>

<span class="tg-spoiler">spoiler</span>

<a href="http://www.example.com/">inline URL</a>
```



<H1>DOUBLE CLICK ME TO FORMAT YOUR TEXT</H1>
<h2>PREVIEW TO YOUR RIGHT</h2>
<H3>When you're done, just paste the entire block into the message_template below</H3>
===================================================================================================================================

<b><u>📝 Test Message</u></b>

<b>Dear {Name},</b>

<b>this is bold</b>

<i>this is italic</i>

<u>underline</u>

<s>strikethrough</s>

<span class="tg-spoiler">this is new!</span> - Click on the moving dots to reveal - <span class="tg-spoiler">spoiler effect</span>


You should not be able to copy or forward this message - do try it out and let us know if you succeed 😜

In [ ]:
# @title <h1>Create Message_Template</h1>

message_template = """<b><u>📝 Test Message</u></b>

<b>Dear {Name},</b>

<b>this is bold</b>

<i>this is italic</i>

<u>underline</u>

<s>strikethrough</s>

<span class="tg-spoiler">this is new!</span> - Click on the moving dots to reveal - <span class="tg-spoiler">spoiler effect</span>


You should not be able to copy or forward this message"""

In [ ]:
your_bot_token = userdata.get('bot_token')

In [ ]:
# @title <h1>Send a preview message</h1>
test_number = "xxxxxxxx" # @param {type:"string"}

# SEND TEST MESSAGE

async def main():

    bot = Bot(token=your_bot_token)

    for index, row in merged_df.iterrows():
      if row['Mobile No'] == test_number:

        row_dict = row.to_dict()
        text = message_template.format(**row_dict)

        await bot.send_message(chat_id=row['telegram_id'], text=text, parse_mode='html', protect_content=True)

# Run the main function
await main()


In [ ]:
# @title <h1>Send Campaign</h1>

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
successful_messages = 0

# Custom rate limiter
class MyRateLimiter(BaseRateLimiter):
    def __init__(self):
        self.lock = asyncio.Lock()
        self.bucket_capacity = 20  # Max 20 tokens (messages)
        self.tokens = 20  # Starts full
        self.last_refill_time = time()
        self.refill_rate = 1 / 20  # 20 messages per second

    async def initialize(self) -> None:
        pass

    async def shutdown(self) -> None:
        pass

    async def process_request(
        self,
        callback,
        args,
        kwargs,
        endpoint,
        data,
        rate_limit_args
    ):
      async with self.lock:
        current_time = time()
        time_elapsed = current_time - self.last_refill_time
        self.last_refill_time = current_time
        self.tokens = min(self.bucket_capacity, self.tokens + time_elapsed / self.refill_rate)

        logger.info(f"Tokens available: {self.tokens}")

        if self.tokens < 1:
            sleep_time = (1 - self.tokens) * self.refill_rate
            logger.info(f"Sleeping for {sleep_time} seconds")
            await sleep(sleep_time)

        self.tokens -= 1
      return await callback(*args, **kwargs)

# Async function to send messages
async def send_message(bot, row):
    global successful_messages
    row_dict = row.to_dict()
    text = message_template.format(**row_dict)
    try:
        await bot.send_message(chat_id=row['telegram_id'], text=text, parse_mode='html', protect_content=True)
        successful_messages += 1
    except telegram.error.BadRequest as e:
        logger.error(f"BadRequest error for {row['Name']}, {row['Mobile No']}: {e}")
    except telegram.error.TelegramError as e:
        logger.error(f"Telegram error for {row['Name']}, {row['Mobile No']}: {e}")
    except Exception as e:
        logger.error(f"An unexpected error occurred for {row['Name']}, {row['Mobile No']}: {e}")

custom_request = HTTPXRequest(
    connection_pool_size=25,
    read_timeout=10.0,
    write_timeout=10.0,
    connect_timeout=10.0,
    pool_timeout=10.0,
    http_version='1.1'
)

# Initialize application builder
application = ApplicationBuilder().token(your_bot_token).rate_limiter(MyRateLimiter()).request(custom_request).build()
bot = application.bot

async def main():
    batch_size = 20  # Set the size of each batch
    for i in range(0, len(merged_df), batch_size):
        # Create a batch of tasks
        batch = merged_df[i:i + batch_size]
        tasks = [asyncio.ensure_future(send_message(bot, row)) for index, row in batch.iterrows()]
        # Wait for the entire batch of tasks to complete before starting the next
        await asyncio.gather(*tasks)
        logger.info(f"Batch {i // batch_size + 1} completed")

    print(f"Total successful messages sent: {successful_messages}")

# Run the main function
await main()

In [ ]:
# @title <h1>View List of Subscribers</h1>
merged_df

In [ ]:
# @title <h1>View List of Non-Subscribers</h1>
merged_outer = pd.merge(df_recipient_list, subscriber_list, on='Mobile No', how='outer', indicator=True)
non_subscribers = merged_outer[merged_outer['_merge'] == 'left_only']
non_subscribers